In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [41]:
import os 
# kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
# spark_k = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"
kafka_spark = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {}  pyspark-shell".format(kafka_spark))

In [42]:
spark = SparkSession.builder.master("local").appName("process-tweet").getOrCreate()

In [43]:
spark

In [44]:
KAFKA_BROKER = "localhost:9092"
KAFKA_TOPIC = "twitter-sentiment"


In [45]:
df = spark.readStream.\
        format("kafka").\
        option("kafka.bootstrap.servers",KAFKA_BROKER).\
        option("subscribe",KAFKA_TOPIC).\
        option("startingOffsets","earliest").\
        load()

In [46]:
tweet = df.selectExpr("CAST(value as string)")

In [47]:
tweet.writeStream.outputMode("append").format("console").start()


In [48]:
#filter all hashtag in each tweet by timestamp
processed_tweet = df.selectExpr("CAST(value as string) AS value").\
                    select(from_json("value","id STRING,time STRING,text STRING,sentiment_score STRING").alias("x")).\
                    selectExpr("CAST(from_unixtime(x.time,'HH:mm:ss') as TIMESTAMP) AS time","explode(split(lower(x.text),' ')) as hashtag").\
                    where("length(hashtag) > 1").\
                    where("substring(hashtag,1,1) = '#'").\
                    selectExpr("(time,hashtag) as result").\
                    select(to_json("result").alias("value"))

In [53]:
processed_tweet_2 = df.selectExpr("CAST(value as string) AS value").\
                    select(from_json("value","id STRING,time STRING,text STRING,sentiment_score STRING").alias("x"))


In [49]:
#count hashtag appears in a range of timestamp
processed_tweet_wdtime = processed_tweet.\
                        select(from_json("value","time STRING,hashtag STRING").alias("x")).\
                        selectExpr("x.time as time","x.hashtag as hashtag").\
                        groupBy("hashtag",window("time",windowDuration = "60 seconds")).agg(count("hashtag").alias("count"))
                                        

In [50]:
processed_tweet.writeStream.\
                outputMode("append").\
                format("console").\
                option("truncate","false").\
                start()

In [51]:
processed_tweet_wdtime.writeStream.\
                outputMode("update").\
                format("console").\
                option("truncate","false").\
                start()

In [55]:
processed_tweet_2.writeStream.\
                outputMode("update").\
                format("console").\
                option("truncate","false").\
                start()

In [56]:
spark.stop()